In [ ]:
import cv2
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import (Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, 
                          Dense, Flatten, Dropout)
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau , ModelCheckpoint, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tqdm import tqdm

In [ ]:
base_path = '../input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/'
image_gen = ImageDataGenerator(rescale=1./255.)

train_flow = image_gen.flow_from_directory(
    base_path + 'train/',
    target_size=(256, 256),
    batch_size=64,
    class_mode='binary'
)

valid_flow = image_gen.flow_from_directory(
    base_path + 'valid/',
    target_size=(256, 256),
    batch_size=64,
    class_mode='binary'
)

test_flow = image_gen.flow_from_directory(
    base_path + 'test/',
    target_size=(256, 256),
    batch_size=64,
    class_mode='binary'
)

In [ ]:
def build_model(pretrained):
    model = Sequential([
        pretrained,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        loss='binary_crossentropy',
        optimizer = Adam(learning_rate = 0.001),
        metrics=['accuracy']
    )
    
    return model

In [ ]:
checkpoint = ModelCheckpoint(filepath='checkpoint.h5',
                             save_best_only=True,
                             verbose=1,
                             mode='min',
                             moniter='val_loss'
                            )
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=3, 
                              verbose=1, 
                              min_delta=0.0001
                             )
csv_logger = CSVLogger('training.log')

callbacks = [checkpoint, reduce_lr, csv_logger]

In [ ]:
efficientnet = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(256, 256, 3),
)
model = build_model(efficientnet)
model.summary()

In [ ]:
train_steps = 100000//64
valid_steps = 20000//64

model.fit(
    train_flow,
    epochs=10,
    callbacks = callbacks,
    steps_per_epoch=train_steps,
    validation_data=valid_flow,
    validation_steps=valid_steps
)

In [ ]:
model.save('./deepfake_models')

In [ ]:
# model = load_model('../input/trainedmodel/deepfake_models/')
# model.summary()

In [ ]:
_, accu = model.evaluate(test_flow)
print('Final Test Acccuracy = {:.3f}'.format(accu*100))

In [ ]:
import PIL
from PIL import Image
im1 = Image.open('../input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test/real/00001.jpg')
p1 = np.array(im1)
plt.imshow(p1)
p1 = np.expand_dims(p1, axis=0)
p1.shape

In [ ]:
preds = model.predict(p1)
prediction = np.argmax(preds)
pct = np.max(preds)
if prediction == 0:
    print('The Prediction of the sample is: It Is ALL')
else:
    print('Prediction is Hem')
print("Prediction Confidence Percentage is: ", pct*100)
plt.imshow(im1);

In [ ]:
print(preds)
print(prediction)

In [ ]:
from lime.wrappers.scikit_image import SegmentationAlgorithm
from skimage.segmentation import mark_boundaries
 
segmentation_fn = SegmentationAlgorithm(
    'slic',
    kernel_size=4,
    max_dist=200,
    ratio=0.2,
    random_seed=42
)

segments = segmentation_fn(im1)

print(len(np.unique(segments)))

plt.imshow(segments)
plt.colorbar();

In [ ]:
from lime import lime_image

explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(np.array(im1), model.predict,  
                                         top_labels=3, hide_color=0, num_samples=1000)

In [ ]:
from skimage.segmentation import mark_boundaries


temp_1, mask_1 = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=40, hide_rest=True)
temp_2, mask_2 = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=40, hide_rest=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,15))
ax1.imshow(mark_boundaries(temp_1, mask_1))
ax2.imshow(mark_boundaries(temp_2, mask_2));

In [ ]:
y_pred = model.predict(test_flow)
y_test = test_flow.classes

In [ ]:
print("ROC AUC Score:", metrics.roc_auc_score(y_test, y_pred))
print("AP Score:", metrics.average_precision_score(y_test, y_pred))
print()
print(metrics.classification_report(y_test, y_pred > 0.5))